# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [1]:
import pandas as pd
import numpy as np

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [5]:

# Cargar el archivo con los argumentos adicionales
loans = pd.read_csv(
    'https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true',
    compression='gzip',       # Descomprime el archivo gzip
    on_bad_lines='skip',      # Ignora las líneas defectuosas
    low_memory=False          # Optimización para archivos grandes
)

# Mostrar el resultado
loans.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [6]:
print(loans.dtypes)

Unnamed: 0                 int64
id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 151, dtype: object


## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [7]:


datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [8]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [9]:
# Guardar el DataFrame en un archivo pickle
datos_dict.to_pickle("datos_dict.pkl")
print("DataFrame guardado en formato pickle.")

DataFrame guardado en formato pickle.


In [10]:
# Cargar el DataFrame desde el archivo pickle
datos_dict_cargado = pd.read_pickle("datos_dict.pkl")
print("DataFrame cargado desde el archivo pickle.")

DataFrame cargado desde el archivo pickle.


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [13]:
# Assuming your DataFrame is already loaded as 'datos_dict'
df = pd.DataFrame(loans)

unique_dtypes = df.dtypes.unique()

print("Unique Data Types in the DataFrame:")
for dtype in unique_dtypes:
    print(dtype)

Unique Data Types in the DataFrame:
int64
float64
object


In [14]:
# Función para procesar columnas int64 y mostrar estadísticas detalladas
def process_int64_column(df, column):
    """
    Procesa una columna de tipo int64 en el DataFrame, realiza conversiones,
    calcula estadísticas y muestra los resultados.

    Parámetros:
    - df: DataFrame de pandas.
    - column: Nombre de la columna a procesar.
    """
    print(f"\n--- Procesando columna: {column} ---")

    # Convertir a tipo numérico, ignorando errores (valores no numéricos -> NaN)
    df[column] = pd.to_numeric(df[column], errors='coerce')

    # Eliminar valores NaN para mantener el tipo entero
    df[column].dropna(inplace=True)

    # Calcular estadísticas básicas
    stats = {
        'Max': df[column].max(),
        'Min': df[column].min(),
        'Average': df[column].mean(),
        'Std Dev': df[column].std()
    }

    # Calcular cuantiles (percentiles 25, 50 (mediana), y 75)
    quantiles = df[column].quantile([0.25, 0.5, 0.75]).to_dict()
    stats.update({
        '25th Percentile': quantiles[0.25],
        'Median (50th Percentile)': quantiles[0.5],
        '75th Percentile': quantiles[0.75]
    })

    # Mostrar los resultados
    for stat, value in stats.items():
        print(f"{stat}: {value:.2f}")
    print(f"--- Fin del procesamiento de la columna: {column} ---\n")


# Obtener la lista de columnas con tipo de dato int64
int64_columns = [col for col in df.columns if df[col].dtype == 'int64']

# Procesar cada columna int64 encontrada
if int64_columns:
    print("Iniciando procesamiento de columnas int64 en el DataFrame...\n")
    for column in int64_columns:
        process_int64_column(df, column)
    print("Procesamiento completo de todas las columnas int64.")
else:
    print("No se encontraron columnas con tipo de dato 'int64' en el DataFrame.")

Iniciando procesamiento de columnas int64 en el DataFrame...


--- Procesando columna: Unnamed: 0 ---
Max: 99999.00
Min: 0.00
Average: 49999.50
Std Dev: 28867.66
25th Percentile: 24999.75
Median (50th Percentile): 49999.50
75th Percentile: 74999.25
--- Fin del procesamiento de la columna: Unnamed: 0 ---


--- Procesando columna: id ---
Max: 38098114.00
Min: 57167.00
Average: 30299954.28
Std Dev: 4763499.79
25th Percentile: 27370147.00
Median (50th Percentile): 30525558.50
75th Percentile: 34382007.00
--- Fin del procesamiento de la columna: id ---

Procesamiento completo de todas las columnas int64.


In [15]:
# Paso siguiente: Procesamiento de columnas categóricas
# Especificamos las columnas que consideramos categóricas
categorical_columns = ["loan_status", "grade", "sub_grade"]  # Ejemplo de columnas categóricas

# Procesar cada columna categórica especificada
for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].astype("category")
        print(f"Columna '{col}' convertida a tipo 'category'.")
    else:
        print(f"Advertencia: La columna '{col}' no se encuentra en el DataFrame y no pudo ser convertida.")

# Justificación: Convertir columnas a `category` ayuda a optimizar el espacio en memoria y facilita su tratamiento
# como datos discretos, especialmente útil para columnas con valores repetitivos o pocos valores únicos.

# Guardar el DataFrame procesado en formato pickle para futuras cargas sin pérdida de tipos de datos
df.to_pickle("processed_data.pkl")
print("\nDataFrame guardado en formato `pickle` como 'processed_data.pkl'.")

# Cargar el DataFrame desde el archivo pickle para verificar
loaded_df = pd.read_pickle("processed_data.pkl")
print("\nDataFrame cargado desde 'processed_data.pkl'. Verificación completa.")

# Crear la tabla `column_types` para registrar los tipos de datos actualizados
column_types = pd.DataFrame({
    'column_name': df.columns,
    'type': df.dtypes
}).reset_index(drop=True)

# Mostrar la tabla `column_types` para revisión
print("\nTabla de tipos de columnas después de los procesamientos:")
print(column_types)

# Justificación final: Crear esta tabla permite tener un registro claro de los tipos de datos
# en cada columna del DataFrame, facilitando la referencia y comprobación para futuras transformaciones o análisis.


Columna 'loan_status' convertida a tipo 'category'.
Columna 'grade' convertida a tipo 'category'.
Columna 'sub_grade' convertida a tipo 'category'.

DataFrame guardado en formato `pickle` como 'processed_data.pkl'.

DataFrame cargado desde 'processed_data.pkl'. Verificación completa.

Tabla de tipos de columnas después de los procesamientos:
               column_name     type
0               Unnamed: 0    int64
1                       id    int64
2                member_id  float64
3                loan_amnt  float64
4              funded_amnt  float64
..                     ...      ...
146      settlement_status   object
147        settlement_date   object
148      settlement_amount  float64
149  settlement_percentage  float64
150        settlement_term  float64

[151 rows x 2 columns]


## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [17]:

import json

# Inicialización del DataFrame de ejemplo (asegúrate de tener tu propio DataFrame)
# df = pd.read_csv('tu_archivo.csv')

# Diccionario para almacenar las estrategias de imputación
imputation_strategies = {}

def register_imputation_strategy(column_name, strategy, value):
    """
    Registra la estrategia de imputación utilizada para una columna específica en un diccionario.
    
    Args:
    - column_name (str): Nombre de la columna.
    - strategy (str): Estrategia de imputación ('median', 'mode', 'drop', 'none').
    - value (various): Valor utilizado para la imputación o razón de eliminación.
    """
    imputation_strategies[column_name] = {
        "estrategia": strategy,
        "valor": value
    }

# 1. Procesamiento de columnas numéricas y categóricas con datos faltantes
for column in df.columns:
    # Solo procesamos columnas que contengan valores NaN
    if df[column].isna().sum() > 0:
        if df[column].dtype in ['float64', 'int64']:  # Columnas numéricas
            missing_ratio = df[column].isna().mean()

            # Menos del 50% de valores faltantes: imputación con la mediana
            if missing_ratio < 0.5:
                median_value = df[column].median()
                df[column].fillna(median_value, inplace=True)
                register_imputation_strategy(column, "median", median_value)

            # Más del 50% de valores faltantes: eliminación de la columna
            else:
                df.drop(columns=[column], inplace=True)
                register_imputation_strategy(column, "drop", "más del 50% de valores faltantes")

        elif df[column].dtype == 'object':  # Columnas categóricas
            missing_ratio = df[column].isna().mean()

            # Menos del 50% de valores faltantes: imputación con el valor más frecuente
            if missing_ratio < 0.5:
                mode_value = df[column].mode()[0]
                df[column].fillna(mode_value, inplace=True)
                register_imputation_strategy(column, "mode", mode_value)

            # Más del 50% de valores faltantes: eliminación de la columna
            else:
                df.drop(columns=[column], inplace=True)
                register_imputation_strategy(column, "drop", "más del 50% de valores faltantes")

    # Columnas sin datos faltantes
    else:
        register_imputation_strategy(column, "none", "sin NaNs")

# Guardar el registro de estrategias de imputación en un archivo JSON
with open("imputation_strategies.json", "w") as f:
    json.dump(imputation_strategies, f, indent=4)

print("Estrategias de imputación guardadas en 'imputation_strategies.json'.")

Estrategias de imputación guardadas en 'imputation_strategies.json'.


In [18]:
# Guardar el DataFrame final en formato parquet
df.to_parquet('data_final.parquet', index=False, compression='gzip')

# Cargar el DataFrame de nuevo para asegurarse de que los datos se guardaron correctamente
df_loaded = pd.read_parquet('data_final.parquet')
print("Datos cargados correctamente:")
print(df_loaded.head())

Datos cargados correctamente:
   Unnamed: 0        id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0           0  38098114    15000.0      15000.0          15000.0   60 months   
1           1  36805548    10400.0      10400.0          10400.0   36 months   
2           2  37842129    21425.0      21425.0          21425.0   60 months   
3           3  37612354    12800.0      12800.0          12800.0   60 months   
4           4  37662224     7650.0       7650.0           7650.0   36 months   

   int_rate  installment grade sub_grade  ... percent_bc_gt_75  \
0     12.39       336.64     C        C1  ...              0.0   
1      6.99       321.08     A        A3  ...             14.3   
2     15.59       516.36     D        D1  ...            100.0   
3     17.14       319.08     D        D4  ...            100.0   
4     13.66       260.20     C        C3  ...            100.0   

  pub_rec_bankruptcies tax_liens  tot_hi_cred_lim total_bal_ex_mort  \
0                  0.

In [22]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Escalar variables numéricas entre 0 y 1
scaler = MinMaxScaler()
df_scaled = df.copy()
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df_scaled[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Estandarizar variables numéricas para media 0 y desviación estándar 1
scaler_standard = StandardScaler()
df_standardized = df.copy()
df_standardized[numeric_cols] = scaler_standard.fit_transform(df[numeric_cols])

# Visualizar datos escalados y estandarizados
print("Datos Escalados (0-1):\n", df_scaled.head())
print("\nDatos Estandarizados (media=0, std=1):\n", df_standardized.head())

Datos Escalados (0-1):
    Unnamed: 0        id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0     0.00000  1.000000   0.411765     0.411765         0.411765   60 months   
1     0.00001  0.966022   0.276471     0.276471         0.276471   36 months   
2     0.00002  0.993271   0.600735     0.600735         0.600735   60 months   
3     0.00003  0.987231   0.347059     0.347059         0.347059   60 months   
4     0.00004  0.988542   0.195588     0.195588         0.195588   36 months   

   int_rate  installment grade sub_grade  ... percent_bc_gt_75  \
0  0.318544     0.222267     C        C1  ...            0.000   
1  0.049352     0.210973     A        A3  ...            0.143   
2  0.478066     0.352716     D        D1  ...            1.000   
3  0.555334     0.209522     D        D4  ...            1.000   
4  0.381854     0.166784     C        C3  ...            1.000   

  pub_rec_bankruptcies tax_liens  tot_hi_cred_lim total_bal_ex_mort  \
0                  0.0     

In [24]:
# Codificación One-Hot para variables categóricas
df_encoded = pd.get_dummies(df, columns=['loan_status', 'grade', 'sub_grade'], drop_first=True)

# Agrupación de categorías (Ejemplo: Simplificar grados)
df['grade_simplified'] = df['grade'].apply(lambda x: 'High' if x in ['A', 'B'] else 'Low')
print(df[['grade', 'grade_simplified']].head())

  grade grade_simplified
0     C              Low
1     A             High
2     D              Low
3     D              Low
4     C              Low


### Codigo para salvar y cargar JSONs

In [25]:
# Convertir tipos de datos para optimizar memoria
df_optimized = df.copy()

# Convertir columnas numéricas pequeñas a int8 o int16
for col in df_optimized.select_dtypes(include='int64').columns:
    df_optimized[col] = pd.to_numeric(df_optimized[col], downcast='integer')

# Convertir columnas float grandes a float32 para reducir memoria
for col in df_optimized.select_dtypes(include='float64').columns:
    df_optimized[col] = pd.to_numeric(df_optimized[col], downcast='float')

# Convertir columnas categóricas en categorías
for col in df_optimized.select_dtypes(include='object').columns:
    df_optimized[col] = df_optimized[col].astype('category')

print("DataFrame optimizado:\n", df_optimized.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 95 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   Unnamed: 0                  100000 non-null  int32   
 1   id                          100000 non-null  int32   
 2   loan_amnt                   100000 non-null  float32 
 3   funded_amnt                 100000 non-null  float32 
 4   funded_amnt_inv             100000 non-null  float32 
 5   term                        100000 non-null  category
 6   int_rate                    100000 non-null  float32 
 7   installment                 100000 non-null  float32 
 8   grade                       100000 non-null  category
 9   sub_grade                   100000 non-null  category
 10  emp_title                   100000 non-null  category
 11  emp_length                  100000 non-null  category
 12  home_ownership              100000 non-null  category
 13  

In [26]:
# Convertir tipos de datos para optimizar memoria
df_optimized = df.copy()

# Convertir columnas numéricas pequeñas a int8 o int16
for col in df_optimized.select_dtypes(include='int64').columns:
    df_optimized[col] = pd.to_numeric(df_optimized[col], downcast='integer')

# Convertir columnas float grandes a float32 para reducir memoria
for col in df_optimized.select_dtypes(include='float64').columns:
    df_optimized[col] = pd.to_numeric(df_optimized[col], downcast='float')

# Convertir columnas categóricas en categorías
for col in df_optimized.select_dtypes(include='object').columns:
    df_optimized[col] = df_optimized[col].astype('category')

print("DataFrame optimizado:\n", df_optimized.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 95 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   Unnamed: 0                  100000 non-null  int32   
 1   id                          100000 non-null  int32   
 2   loan_amnt                   100000 non-null  float32 
 3   funded_amnt                 100000 non-null  float32 
 4   funded_amnt_inv             100000 non-null  float32 
 5   term                        100000 non-null  category
 6   int_rate                    100000 non-null  float32 
 7   installment                 100000 non-null  float32 
 8   grade                       100000 non-null  category
 9   sub_grade                   100000 non-null  category
 10  emp_title                   100000 non-null  category
 11  emp_length                  100000 non-null  category
 12  home_ownership              100000 non-null  category
 13  

In [27]:
# Diccionario para documentar cambios
documentacion = {
    "escalado": "Escalado de variables numéricas a un rango de 0 a 1 usando MinMaxScaler",
    "tipos_optimizados": "Conversiones de columnas a tipos de datos optimizados para ahorrar memoria",
    "imputaciones": "Métodos de imputación específicos para valores faltantes",
    "categorias": "Codificación One-Hot para variables categóricas y agrupación de categorías"
}

# Guardar documentación en un archivo JSON
with open("documentacion_proceso.json", "w") as json_file:
    json.dump(documentacion, json_file, indent=4)

In [23]:
# Función para guardar datos en un archivo JSON
def guardar_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f"Datos guardados exitosamente en {filename}")

# Ejemplo de uso con un diccionario
datos_dict = {
    "columna1": {"estrategia": "media", "valor": 3.4},
    "columna2": {"estrategia": "moda", "valor": "missing"}
}
guardar_json(datos_dict, "datos.json")

Datos guardados exitosamente en datos.json
